# Task 4.8

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import data file "orders_products_combined.pkl"
ords_prods_merge = pd.read_pickle('/Users/marika/Instacart Basket Analysis 04.14.24/02 Data/Prepared Data/orders_products_merged.pkl')

In [3]:
# Check the DataFrame structure
ords_prods_merge.head()

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,Unnamed: 0_y,product_name,aisle_id,department_id,prices,merge_indicator,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,0,2539329,1,1,2,8,NaN,196,1,0,...,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,1,2398795,1,2,3,7,15.0,196,1,1,...,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders
2,2,473747,1,3,3,12,21.0,196,1,1,...,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders
3,3,2254736,1,4,4,7,29.0,196,1,1,...,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders
4,4,431534,1,5,4,15,28.0,196,1,1,...,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders


In [4]:
# Calculate the aggregated mean of the "order_number" column grouped by "department_id" for the entire DataFrame
department_order_number_mean = ords_prods_merge.groupby('department_id')['order_number'].mean()

In [5]:
# Display the aggregated mean
print(department_order_number_mean)

department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.213779
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170828
12    15.887622
13    16.583304
14    16.757377
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


In [ ]:
The results for the entire DataFrame will give us the average order number for each department across all orders made by all users.
In contrast, the results for the subset would have provided the average order number for each department within a specific subset of 
orders.

In [6]:
# Define the threshold for loyalty (e.g., customers with more than 10 orders are considered loyal)
loyalty_threshold = 10

In [7]:
# Create a new column 'loyalty_flag' and initialize with 'Regular' for all customers
ords_prods_merge['loyalty_flag'] = 'Regular'

In [8]:
# Use transform to calculate the total number of orders for each user
ords_prods_merge['total_orders'] = ords_prods_merge.groupby('user_id')['order_number'].transform('max')

In [9]:
# Update the loyalty flag for existing customers
ords_prods_merge.loc[ords_prods_merge['total_orders'] > loyalty_threshold, 'loyalty_flag'] = 'Loyal'

In [10]:
# Drop the temporary column 'total_orders'
ords_prods_merge.drop('total_orders', axis=1, inplace=True)

# Products purchased by regular or new customers

In [11]:
# Create subsets for each loyalty category
loyal_customers = ords_prods_merge[ords_prods_merge['loyalty_flag'] == 'Loyal']
regular_customers = ords_prods_merge[ords_prods_merge['loyalty_flag'] == 'Regular']
new_customers = ords_prods_merge[ords_prods_merge['loyalty_flag'] == 'New']

In [12]:
# Calculate basic statistics of product prices for each loyalty category
loyal_prices_stats = loyal_customers['prices'].describe()
regular_prices_stats = regular_customers['prices'].describe()
new_prices_stats = new_customers['prices'].describe()

In [13]:
# Display the basic statistics for each loyalty category
print("Loyal Customers:")
print(loyal_prices_stats)
print("\nRegular Customers:")
print(regular_prices_stats)
print("\nNew Customers:")
print(new_prices_stats)

Loyal Customers:
count    2.618481e+07
mean     1.166772e+01
std      4.678503e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      9.999900e+04
Name: prices, dtype: float64

Regular Customers:
count    6.249398e+06
mean     1.329437e+01
std      5.973017e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      9.999900e+04
Name: prices, dtype: float64

New Customers:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: prices, dtype: float64


# Mean price of products purchased by each use

In [18]:
# Calculate the mean price of products purchased by each user and reset the index
user_mean_prices = ords_prods_merge.groupby('user_id')['prices'].mean().reset_index()

# Create a spending flag for each user based on the average price
ords_prods_merge['spending_flag'] = 'Low spender'
ords_prods_merge.loc[user_mean_prices[user_mean_prices['prices'] >= 10]['user_id'], 'spending_flag'] = 'High spender'

# Display the first few rows to verify the spending flags
print(ords_prods_merge[['user_id', 'spending_flag']].head())

   user_id spending_flag
0        1   Low spender
1        1   Low spender
2        1   Low spender
3        1   Low spender
4        1   Low spender


# Median of the "days_since_prior_order" column

In [23]:
# Calculate the median of "days_since_prior_order" for each user and reset the index
user_median_days_since_prior_order = ords_prods_merge.groupby('user_id')['days_since_prior_order'].median().reset_index()

In [24]:
# Create a new column 'order_frequency_flag' and initialize it with 'Frequent customer' for all users
ords_prods_merge['order_frequency_flag'] = 'Frequent customer'

In [25]:
# Assign the order frequency flag based on the median value
ords_prods_merge.loc[ords_prods_merge['user_id'].isin(user_median_days_since_prior_order[user_median_days_since_prior_order['days_since_prior_order'] > 20]['user_id']), 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['user_id'].isin(user_median_days_since_prior_order[user_median_days_since_prior_order['days_since_prior_order'] <= 20]['user_id'])) & (ords_prods_merge['user_id'].isin(user_median_days_since_prior_order[user_median_days_since_prior_order['days_since_prior_order'] > 10]['user_id'])), 'order_frequency_flag'] = 'Regular customer'

In [26]:
# Display the first few rows to verify the order frequency flags
print(ords_prods_merge[['user_id', 'order_frequency_flag']].head())

   user_id   order_frequency_flag
0        1  Non-frequent customer
1        1  Non-frequent customer
2        1  Non-frequent customer
3        1  Non-frequent customer
4        1  Non-frequent customer


In [27]:
# Define the path to the directory containing the CSV file
path = '/Users/marika/Instacart Basket Analysis 04.14.24'

# Export data to pkl
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))